In [1]:
#Imports---------------------------------------------------------------
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image
#%matplotlib inline
#%config InlineBackend.figure_format = 'retina'

import glob
import numpy as np
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
#Main pipeline for multiclass classification
#import libraries
import numpy as np
from time import time
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset
from tqdm import tqdm
from sklearn.model_selection import KFold
from torchvision import models, transforms
import pickle
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


Mounted at /content/drive


In [3]:
K=5
arc='alexnet'
loss_func = nn.CrossEntropyLoss()
lr=0.00001
b_s=32
ep=50
PATH='/content/drive/My Drive/ML/model.pth'
#APAGAR-------------------------------------
#with open('/content/drive/My Drive/ML/Resultados/readme.txt', 'w') as f:
#    f.write('')
#with open('/content/drive/My Drive/ML/Resultados/readme.txt', 'a') as f:
#    f.write('------------------------------------------------Self-Supervised\nParametros:\n\nEpochs'+str(ep)+'\nArc:'+arc+'\nLearning rate:'+str(lr)+'\nBatch size:'+str(b_s))

In [4]:
#Standerdize the image----------------------------------------------------------------------
from torchvision.transforms import transforms
#from RandAugment import RandAugment
transform = transforms.Compose([            #[1]
 #transforms.Resize(224),                    #[2]
 #transforms.CenterCrop(224),                #[3]
 transforms.RandomHorizontalFlip(),
 transforms.RandomVerticalFlip(),
 transforms.ToTensor(),                     #[4]
 transforms.Normalize(                      #[5]
 mean=[0.485, 0.456, 0.406],                #[6]
 std=[0.229, 0.224, 0.225]                  #[7]
 )])
 

transform_strong = transforms.Compose([
 #transforms.Resize(224),                    #[2]
 #transforms.CenterCrop(224),                #[3]
 transforms.RandomHorizontalFlip(),
 transforms.RandomVerticalFlip(),
 torchvision.transforms.RandAugment(2,6),
 transforms.ToTensor(),                     #[4]
 transforms.Normalize(                      #[5]
 mean=[0.485, 0.456, 0.406],                #[6]
 std=[0.229, 0.224, 0.225]                  #[7]
 )])
 

In [5]:
infile = open('/content/drive/My Drive/ML/NCI_data.pickle','rb')
data=pickle.load(infile)
infile.close
#print(data)
infile = open('/content/drive/My Drive/ML/data_u.pkl','rb')
data_u=pickle.load(infile)
infile.close
print(data_u[1])
i=0
x=[]
y=[]
while(i<len(data_u)):
 x.append(data_u[i][0])
 y.append(data_u[i][1])
 i=i+1
print(x)
class MyDataset(Dataset):
   def __init__(self, type,datas, transform,strong, path,fold):
       if(type=='unlabeled_w' or type=='unlabeled_s'  ):
         
         self.X,self.Y=x,y
       else:  
         self.X,self.Y=datas[fold][type]
       self.transform = transform
       self.strong = strong
       self.path = path
       
   def __getitem__(self, i):
       PATH=self.path+self.X[i]
       while(True):
         try:
           X = Image.open(PATH)
           break
         except IOError:
           print('erro'+self.X[i])
       if(type=='unlabeled_w'):
         X = self.transform(X)
       elif(type=='unlabeled_s'):
         X = self.strong(X)  
       else:
         X = self.transform(X)
       Y = self.Y[i]
       #if(Y == -2.0):
        # Y=self.strong(X)
         #Y=10
       return X, Y

   def __len__(self):
       return len(self.X)


       


['I803018_C1.jpg', -2.0]
['I772654_C2.jpg', 'I803018_C1.jpg', 'I207709_C1.jpg', 'I313262_C1.jpg', 'I102930_C2.jpg', 'I512104_C2.jpg', 'I242619_C2.jpg', 'I651181_C1.jpg', 'I395462_C2.jpg', 'I758770_C1.jpg', 'I715364_C2.jpg', 'I539319_C1.jpg', 'I681042_C1.jpg', 'I852006_C2.jpg', 'I444562_C1.jpg', 'I449616_C1.jpg', 'I596706_C1.jpg', 'I653681_C2.jpg', 'I594925_C2.jpg', 'I837432_C1.jpg', 'I337658_C2.jpg', 'I163391_C1.jpg', 'I127918_C1.jpg', 'I877256_C2.jpg', 'I426096_C2.jpg', 'I559903_C1.jpg', 'I562072_C1.jpg', 'I349657_C1.jpg', 'I798408_C2.jpg', 'I294985_C1.jpg', 'I687993_C2.jpg', 'I464307_C2.jpg', 'I631035_C2.jpg', 'I726919_C2.jpg', 'I466245_C1.jpg', 'I360554_C2.jpg', 'I773158_C1.jpg', 'I715706_C1.jpg', 'I299657_C1.jpg', 'I873638_C1.jpg', 'I428528_C1.jpg', 'I846039_C1.jpg', 'I377393_C2.jpg', 'I795894_C2.jpg', 'I788813_C1.jpg', 'I372128_C1.jpg', 'I262934_C1.jpg', 'I363154_C2.jpg', 'I244905_C1.jpg', 'I264826_C1.jpg', 'I443754_C1.jpg', 'I559862_C1.jpg', 'I876525_C1.jpg', 'I566946_C1.jpg', 'I

In [6]:
ds= MyDataset('train',data,transform,transform_strong,'/content/drive/My Drive/ML/train/Transform/',1)
tr = DataLoader(ds, b_s, True)
#v_ds= MyDataset('val',transform,'/content/drive/My Drive/ML/CVT_1/Transform',1)
#val = DataLoader(v_ds, b_s, True)
t_ds= MyDataset('test',data,transform,transform_strong,'/content/drive/My Drive/ML/train/Transform/',1)
test = DataLoader(t_ds, b_s, True)
uw_ds= MyDataset('unlabeled_w',data_u,transform,transform_strong,'/content/drive/My Drive/ML/train/Transform/',1)
unl_w = DataLoader(uw_ds, b_s, True)
us_ds= MyDataset('unlabeled_s',data_u,transform,transform_strong,'/content/drive/My Drive/ML/train/Transform/',1)
unl_s = DataLoader(us_ds, b_s, True)


In [ ]:
print(len(tr))
print(len(test))

546
182


In [7]:
# Model
class Base(nn.Module):
    def __init__(self, pretrained_model, n_outputs):
        super().__init__()
        self.n_outputs = n_outputs
        model = getattr(models, pretrained_model)(pretrained=True) #using transfer learning from a pretrained model - True
        model = nn.Sequential(*tuple(model.children())[:-1])
        last_dimension = torch.flatten(model(torch.randn(1, 3, 224, 224))).shape[0]
        self.model = nn.Sequential(
            model,
            nn.Flatten(),
            nn.Dropout(0.4),
            nn.Linear(last_dimension, 512),
            nn.Dropout(0.4),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, K)
        )

    def forward(self, x):
        return self.model(x)

    def loss(self, Yhat, Y):
        return loss_func(Yhat, Y)

    def to_proba(self, Yhat):
        return F.softmax(Yhat, 1)

    def to_classes(self, Phat):
        return Phat.argmax(1)


In [8]:
#Give to the model the architecture and number of classes (K)
model = Base(arc, K)
#put the model on 'GPU' or 'CPU'
model = model.to(device)
#Define optimizer and scheduler 
optimizer = optim.Adam(model.parameters(),lr )

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True)


Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

In [ ]:
import torch.nn.functional as F
gama=0.2
thresh=0.8
running_loss=1010.0
train_acc=0.0
final_t_a=0.0
l_train=546*32
l_test=182*32
avg_acc=0.0
best_loss=1000.0
best_acc=0.0
torch.cuda.empty_cache()
best_acc_t=0
best_loss_t=0
PATH2='/content/drive/My Drive/ML/files/unl.pth'
labeled_iter = iter(tr)
unlabeledw_iter = iter(unl_w)
unlabeleds_iter = iter(unl_s)



for epoch in range(ep):
    print('\n')
    print('Iteração:',epoch)
    print('loss:',running_loss)
    print('acc:',avg_acc)
    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': running_loss
            
            }, PATH2)
    end=0
    count=0
    while (end==0):
      count=count+1
      if(count==546):
        end=1
     
      
      inputs, labels = labeled_iter.next()
      weak,_ = unlabeledw_iter.next()
      strong,_ = unlabeleds_iter.next()


      inputs,labels = inputs.to(device),labels.to(device)
      weak,strong = weak.to(device),strong.to(device)

      #labeled data---------------------------------------------
      outputs = model(inputs)  
      Lx = F.cross_entropy(outputs, labels, reduction='mean')    
      loss_l = loss_func(outputs, labels)
      _, predicted = torch.max(outputs, 1)
      real=labels
      correct = (predicted == real).float().sum()
      avg_acc +=correct*100/(len(inputs)*32)

           
      #Unlabeled data----------------------------------------------------
      outputs = model(weak)
      outputs_s = model(strong)
      pseudo_label =  torch.softmax(outputs,dim=-1)
      pseudo_label,index = torch.max(pseudo_label,dim=-1)
      mask = pseudo_label.ge(thresh).float()
      Lu = (F.cross_entropy(outputs_s, index,reduction='none') * mask).mean()
      #loss_u = loss_func(outputs, index)*mask
     
      #Calculate both losses------------------------------------------------
      loss = Lx + gama*Lu
      #loss = loss_l + gama*loss_u
      loss.backward()
      optimizer.step()     
      running_loss += loss.item()
      print('.',end='')

      if avg_acc>=best_acc and running_loss<=best_loss :
             best_acc=avg_acc
             best_loss=running_loss
             




Iteração: 0
loss: 1010.0
acc: 0.0
erroI303088_C1.jpg
erroI303088_C1.jpg
erroI303088_C1.jpg
.......................................................................

In [ ]:
running_loss=1010.0
train_acc=0.0
final_t_a=0.0
l_train=546*32
l_test=182*32
avg_acc=0.0
best_loss=1000.0
best_acc=0.0
torch.cuda.empty_cache()
best_acc_t=0
best_loss_t=0

##Train the dataset
for epoch in range(ep):  # loop over the dataset multiple times
        print('\n')
        print('Iteração:',epoch)
        print('loss:',running_loss)
        print('acc:',avg_acc)
        running_loss2=0.0
        train_acc=0.0
        final_t_a=0.0
        avg_acc2=0.0
        
        ##Test the dataset
        for inputs,labels in test:
            # get the inputs; data is a list of [inputs, labels]
            for x,y in iputs,labels:
              if(y>-2):
                inputs_l.append(x)
                labels_l.appens(y)
              else:
                inputs_u.append(x)
                labels_u.append(y)
            inputs_l, labels_l = inputs_l.to(device),labels_l.to(device)
            labels_l = labels_l.type(torch.LongTensor)
            labels_l=labels_l.to(device)
            inputs_u, labels_u = inputs_u.to(device),labels_u.to(device)
            labels_u = labels_u.type(torch.LongTensor)
            labels_u=labels_u.to(device)
            # forward + backward + optimize
            outputs =model(inputs_l)
            
            
            #print(outputs)
            loss2 = loss_func(outputs, labels_l)
            _, predicted = torch.max(outputs, 1)
            real=labels_l
            correct = (predicted == real).float().sum()
            avg_acc2 +=correct*100/l_test
            print('.',end='')
           
            running_loss2 += loss2.item()

            outputs =model(inputs_u)

           
        print('\n')
        print('loss:',running_loss2)
        print('acc:',avg_acc2)
        print('Finished Validation')

        if avg_acc2>=best_acc and running_loss2<=best_loss :
             best_acc=avg_acc2
             best_loss=running_loss2
             torch.save(model.state_dict(), PATH)
             print('saved')
     

        running_loss = 0.0
        avg_acc = 0        
        for inputs,labels in tr:
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = inputs.to(device),labels.to(device)
            labels = labels.type(torch.LongTensor)
            labels=labels.to(device)
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs =model(inputs)
            #print(outputs)
            loss = loss_func(outputs, labels)
            loss.backward()
            optimizer.step()
            
            #print(outputs)
            #print(labels)
            _, predicted = torch.max(outputs, 1)
            real=labels
            correct = (predicted == real).float().sum()
            avg_acc +=correct*100/l_train
            print('.',end='')
           
            running_loss += loss.item()
            if avg_acc>=best_acc and running_loss<=best_loss :
             best_acc_t=avg_acc
             best_loss_t=running_loss
             
            
           

print('Finished Training')


with open('/content/drive/My Drive/ML/Resultados/readme.txt', 'a') as f:
    f.write('------------------------------------------------\nResultados:\n\nLoss treino:'+str(best_loss_t)+'\nAcc treino:'+str(best_acc_t)+'\nLoss Test:'+str(best_loss)+'\nAcc test:'+str(best_acc))




Iteração: 0
loss: 1010.0
acc: 0.0


OSError: ignored

In [ ]:
model = Base(arc, K)
#put the model on 'GPU' or 'CPU'
model = model.to(device)
model.load_state_dict(torch.load(PATH))
model.eval()

#evaluate model performance
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

y_pred=predict_proba(val)
Phat = []

for inputs,labels in val:
            y_true=labels
            Phat += list(y_true.cpu().numpy())


print(Phat)
print(y_pred)
print('Classification Report:\n', classification_report(Phat, y_pred))

print('Confusion Matrix:')
cm = sns.heatmap(confusion_matrix(Phat, y_pred), annot=True, cmap='Blues')
plt.show()